In [157]:
import tensorflow as tf
import numpy as np
import keras

In [158]:
learning_rate = 0.001
batch_size = 128
display_step = 10

In [159]:
num_input = 32*32
num_classes = 2
dropout = 0.75

X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

In [161]:
import pickle
f = open('dataSet.pickle', 'r')
dataset = pickle.load(f)
f.close()
x_train, y_train, x_test, y_test = dataset["x_train"][:17000], dataset["y_train"][:17000], dataset["x_test"][:4000], dataset["y_test"][:4000]
dataset = None

In [162]:
x_train1 = np.zeros((x_train.shape[0], 32*32))
x_test1 = np.zeros((x_test.shape[0], 32*32))

for i in range(x_train.shape[0]):
    x_train1[i] = np.reshape(x_train[i], (32*32))

for i in range(x_test.shape[0]):
    x_test1[i] = np.reshape(x_test[i], (32*32))
    
x_train1 = (x_train1-128)/255.0
x_test1 = (x_test1-128)/255.5
x_train = None
x_test = None

In [163]:
y_train1 = keras.utils.to_categorical(y_train, num_classes)
y_test1 = keras.utils.to_categorical(y_test, num_classes)


In [164]:
print x_train1.shape
print x_test1.shape
print y_train1.shape
print y_test1.shape

(17000, 1024)
(4000, 1024)
(17000, 2)
(4000, 2)


In [165]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

# Create model
def conv_net(x, weights, biases, dropout):
    x = tf.reshape(x, shape=[-1, 32, 32, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Convolution Layer
    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
    # Max Pooling (down-sampling)
    conv3 = maxpool2d(conv3, k=2)
    
    # Fully connected layer 1
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    
    # Fully connected layer 2
    # Reshape conv2 output to fit fully connected layer input
    fc2 = tf.reshape(fc1, [-1, weights['wd2'].get_shape().as_list()[0]])
    fc2 = tf.add(tf.matmul(fc2, weights['wd2']), biases['bd2'])
    fc2 = tf.nn.relu(fc2)
    
    # Apply Dropout
    fc2 = tf.nn.dropout(fc2, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc2, weights['out']), biases['out'])
    return out

In [168]:
weights = {
    # 5x5 conv, 1 input, 100 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 50])),
    # 5x5 conv, 32 inputs, 200 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 50, 100])),
    # 5x5 conv, 64 inputs, 400 outputs
    'wc3': tf.Variable(tf.random_normal([5, 5, 100, 200])),
    # fully connected, 4*4*400 inputs, 3200 outputs
    'wd1': tf.Variable(tf.random_normal([4*4*200, 1600])),
    # fully connected, 3200 inputs, 1600 outputs
    'wd2': tf.Variable(tf.random_normal([1600, 800])),
    # 800 inputs, 2 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([800, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([50])),
    'bc2': tf.Variable(tf.random_normal([100])),
    'bc3': tf.Variable(tf.random_normal([200])),
    'bd1': tf.Variable(tf.random_normal([1600])),
    'bd2': tf.Variable(tf.random_normal([800])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

# Construct model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [169]:
epoch = 10
batch_size = 50

accuV = []
lossV = []

with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    
    for step in range(1, epoch+1):
        for start in range(0, x_train1.shape[0]-50, 50):
            batch_x, batch_y = x_train1[start:start+50], y_train1[start:start+50]
            sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: 0.8})
        
        
            if start % 1000 == 0:
            # Calculate batch loss and accuracy
                loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                     Y: batch_y,
                                                                         keep_prob: 1.0})
                lossV.append(loss)
                accuV.append(acc)
                print("Epoch " + str(step) + " Pos: "+ str(start) + ", Minibatch Loss= " + \
                      "{:.4f}".format(loss) + ", Training Accuracy= " + \
                      "{:.3f}".format(acc))
        if(step%10==0):
            print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: x_test1[:1000],
                                      Y: y_test1[:1000],
                                      keep_prob: 1.0}))
    print("Optimization Finished!")

    # Calculate accuracy for 256 MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: x_test1[:1000],
                                      Y: y_test1[:1000],
                                      keep_prob: 1.0}))

Epoch 1 Pos: 0, Minibatch Loss= 61662480.0000, Training Accuracy= 0.440
Epoch 1 Pos: 1000, Minibatch Loss= 7874937.0000, Training Accuracy= 0.620
Epoch 1 Pos: 2000, Minibatch Loss= 5805683.0000, Training Accuracy= 0.540
Epoch 1 Pos: 3000, Minibatch Loss= 7494966.5000, Training Accuracy= 0.680
Epoch 1 Pos: 4000, Minibatch Loss= 4963114.0000, Training Accuracy= 0.680
Epoch 1 Pos: 5000, Minibatch Loss= 7454002.0000, Training Accuracy= 0.540
Epoch 1 Pos: 6000, Minibatch Loss= 3109427.5000, Training Accuracy= 0.700
Epoch 1 Pos: 7000, Minibatch Loss= 2123287.7500, Training Accuracy= 0.720
Epoch 1 Pos: 8000, Minibatch Loss= 2658796.7500, Training Accuracy= 0.640
Epoch 1 Pos: 9000, Minibatch Loss= 2934016.7500, Training Accuracy= 0.660
Epoch 1 Pos: 10000, Minibatch Loss= 3012227.5000, Training Accuracy= 0.600
Epoch 1 Pos: 11000, Minibatch Loss= 2169099.0000, Training Accuracy= 0.620
Epoch 1 Pos: 12000, Minibatch Loss= 1046491.5000, Training Accuracy= 0.720
Epoch 1 Pos: 13000, Minibatch Loss= 1

Epoch 7 Pos: 12000, Minibatch Loss= 520.4763, Training Accuracy= 0.580
Epoch 7 Pos: 13000, Minibatch Loss= 430.3825, Training Accuracy= 0.720
Epoch 7 Pos: 14000, Minibatch Loss= 186.3086, Training Accuracy= 0.620
Epoch 7 Pos: 15000, Minibatch Loss= 483.5944, Training Accuracy= 0.640
Epoch 7 Pos: 16000, Minibatch Loss= 474.5825, Training Accuracy= 0.600
Epoch 8 Pos: 0, Minibatch Loss= 1408.4882, Training Accuracy= 0.580
Epoch 8 Pos: 1000, Minibatch Loss= 1000.5172, Training Accuracy= 0.600
Epoch 8 Pos: 2000, Minibatch Loss= 414.8400, Training Accuracy= 0.520
Epoch 8 Pos: 3000, Minibatch Loss= 249.2224, Training Accuracy= 0.640
Epoch 8 Pos: 4000, Minibatch Loss= 180.1855, Training Accuracy= 0.560
Epoch 8 Pos: 5000, Minibatch Loss= 481.6108, Training Accuracy= 0.500
Epoch 8 Pos: 6000, Minibatch Loss= 252.2970, Training Accuracy= 0.600
Epoch 8 Pos: 7000, Minibatch Loss= 118.4683, Training Accuracy= 0.640
Epoch 8 Pos: 8000, Minibatch Loss= 236.4473, Training Accuracy= 0.600
Epoch 8 Pos: 900